<a href="https://colab.research.google.com/github/viniciusrpb/cic0193_machinelearning/blob/main/cap3_1_preprocessamento_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Capítulo 3

## 3.1. Pré-processamento de dados tabulares

Informações sobre o conjunto de dados podem ser acessadas aqui:

https://archive.ics.uci.edu/ml/datasets/Codon+usage#

É um conjunto de dados que pode ser utilizado para classificar microorganismos (vírus, bactérias, protozoários etc) em reino conforme suas características de sequenciamento de RNA.

In [39]:
import pandas as pd
import numpy as np
from sklearn import preprocessing

Primeiro, carregamos o arquivo .CSV associado ao conjunto de dados para um DataFrame:

In [40]:
rawData = pd.read_csv('/datasets/tabular/codon_usage.csv')

Apresentação dos dados em um modelo tabular:

In [41]:
rawData

,Kingdom,DNAtype,SpeciesID,Ncodons,SpeciesName,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,vrl,0.0,100217.0,1995.0,Epizootic haematopoietic necrosis virus,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,vrl,0.0,100220.0,1474.0,Bohle iridovirus,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,vrl,0.0,100755.0,4862.0,Sweet potato leaf curl virus,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,vrl,0.0,100880.0,1915.0,Northern cereal mosaic virus,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,vrl,0.0,100887.0,22831.0,Soil-borne cereal mosaic virus,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13024,pri,0.0,9601.0,1097.0,Pongo pygmaeus abelii,0.02552,0.03555,0.00547,0.01367,0.01276,0.02097,0.00820,0.03555,0.01459,0.03920,0.01003,0.02097,0.00912,0.02005,0.01003,0.03737,0.02279,0.01367,0.01094,0.00000,0.01641,0.01094,0.01185,0.00182,0.01094,0.00456,0.01276,0.01094,0.01276,0.02097,0.01003,0.01003,0.00091,0.01732,0.01459,0.01276,0.01276,0.01641,0.00820,0.02370,0.02097,0.02917,0.03464,0.02005,0.02005,0.01459,0.02461,0.01367,0.01459,0.01914,0.03008,0.00182,0.00547,0.00547,0.00820,0.01367,0.01094,0.01367,0.02279,0.02005,0.04102,0.00091,0.00091,0.00638
13025,pri,1.0,9601

### Análise do conjunto de dados e seus atributos

Aqui podemos ver a quantidade de instâncias x quantidade de atributos dos dados originais:

In [42]:
print(rawData.shape)

(13029, 69)


Em seguida, estamos analisando os tipos de cada atributo do conjunto de dados. É importante observar que atributos numéricos devem estar representados como float64, atributos categóricos como categoric ou int.

In [43]:
rawData.dtypes

Kingdom         object
DNAtype        float64
SpeciesID      float64
Ncodons        float64
SpeciesName     object
                ...   
GAA            float64
GAG            float64
UAA            float64
UAG            float64
UGA            float64
Length: 69, dtype: object

Após uma inspeção visual, vemos que o atributo *"DNAtype"* é do tipo categórico. Vamos corrigir seu tipo utilizando os comandos:

In [44]:
rawData['DNAtype'] = rawData['DNAtype'].astype(object)

rawData.dtypes

Kingdom         object
DNAtype         object
SpeciesID      float64
Ncodons        float64
SpeciesName     object
                ...   
GAA            float64
GAG            float64
UAA            float64
UAG            float64
UGA            float64
Length: 69, dtype: object

Vamos analisar a quantidade de valores que os atributos "Kingdom" (reino) e "SpeciesName" (Nome da espécie) possuem
:

In [45]:
print(rawData['Kingdom'].value_counts())

bct    2920
vrl    2832
pln    2523
vrt    2077
inv    1345
mam     572
phg     220
rod     215
pri     180
arc     126
plm      18
-         1
Name: Kingdom, dtype: int64


In [46]:
print(rawData['SpeciesName'].value_counts())

Escherichia coli O157                         4
Escherichia coli O111                         3
Cotton leaf curl Rajasthan virus - [India     3
Ageratum yellow vein Hualian virus-[Taiwan    3
Squash leaf curl China virus - [Pumpkin       2
                                             ..
mitochondrion Austrofundulus transilis        1
Cercospora nicotianae                         1
Limonium bicolor                              1
Streptococcus phage 700P1                     1
Banana bunchy top virus                       1
Name: SpeciesName, Length: 13017, dtype: int64


### Filtragem de instâncias

A seguir, vamos realizar uma filtragem de instâncias para amostrar apenas aquelas pertencentes aos reinos "bct", "vrl", "pln" e "vrt":

In [47]:
flag1 = rawData['Kingdom'] == 'bct'
flag2 = rawData['Kingdom'] == 'vrl'
flag3 = rawData['Kingdom'] == 'pln'
flag4 = rawData['Kingdom'] == 'vrt'

df = rawData[ flag1 | flag2 | flag3 | flag4]

print(df.shape)

(10352, 69)


### Seleção manual de atributos

Suponha que queremos classificar os microorganismos conforme o reino (Kingdom). Precisamos separar o atributo associado como o atributo classe e remover os atributos SpeciesID e SpeciesName pois eles não são importantes devido à sua granulidade.

In [48]:
categories = df['Kingdom']

species = df['SpeciesID']

df = df.drop(['SpeciesName','SpeciesID','Kingdom'],axis=1)

### Remoção de instâncias com valores ausentes

É comum que dados ausentes se apresentem como valores em branco " ", interrogação "?" ou hífen "-". Substituímos esses caracteres por NaN e depois utilizamos o método dropna do DataFrame:

In [49]:
df = df.replace('-',np.NaN)
df = df.replace(' ',np.NaN)
df = df.replace('?',np.NaN)
df = df.dropna(axis=0)

print(df.shape)

(10350, 66)


### Transformação do espaço de atributos

Como o atributo "DNAtype" é categórico, temos que transformá-lo para atributos numéricos para que os dados processados possam ser utilizados como entrada pelos modelos de aprendizado de máquina (em sua maioria, só aceitam dados em formato estruturado e numérico).

A estratégia é converter o atributo "DNAtype" para binário por meio de variáveis Dummy.

In [50]:
df = pd.get_dummies(df)

df

,DNAtype,Ncodons,UUU,UUC,UUA,UUG,CUU,CUC,CUA,CUG,AUU,AUC,AUA,AUG,GUU,GUC,GUA,GUG,GCU,GCC,GCA,GCG,CCU,CCC,CCA,CCG,UGG,GGU,GGC,GGA,GGG,UCU,UCC,UCA,UCG,AGU,AGC,ACU,ACC,ACA,ACG,UAU,UAC,CAA,CAG,AAU,AAC,UGU,UGC,CAU,CAC,AAA,AAG,CGU,CGC,CGA,CGG,AGA,AGG,GAU,GAC,GAA,GAG,UAA,UAG,UGA
0,0.0,1995.0,0.01654,0.01203,0.00050,0.00351,0.01203,0.03208,0.00100,0.04010,0.00551,0.02005,0.00752,0.02506,0.01103,0.04110,0.00902,0.03308,0.01003,0.05013,0.01554,0.01103,0.02356,0.03208,0.01203,0.00501,0.01003,0.01203,0.03158,0.01905,0.02456,0.01353,0.02155,0.00251,0.00652,0.00150,0.01554,0.00501,0.02105,0.00902,0.01053,0.00501,0.02256,0.00301,0.03108,0.00401,0.02607,0.00251,0.01153,0.00501,0.02356,0.01053,0.03860,0.00401,0.00702,0.00401,0.00451,0.01303,0.03559,0.01003,0.04612,0.01203,0.04361,0.00251,0.00050,0.00000
1,0.0,1474.0,0.02714,0.01357,0.00068,0.00678,0.00407,0.02849,0.00204,0.04410,0.01153,0.02510,0.00882,0.03324,0.00814,0.04071,0.00814,0.03256,0.01085,0.04885,0.01221,0.01357,0.00678,0.02714,0.01221,0.00407,0.01425,0.01221,0.01967,0.02239,0.01289,0.02103,0.01493,0.00407,0.00475,0.00068,0.02035,0.00950,0.02782,0.01425,0.00611,0.00475,0.02917,0.00407,0.02374,0.00882,0.02917,0.00271,0.01628,0.00204,0.01967,0.00543,0.03392,0.00136,0.00678,0.00136,0.00136,0.01696,0.03596,0.01221,0.04545,0.01560,0.04410,0.00271,0.00068,0.00000
2,0.0,4862.0,0.01974,0.02180,0.01357,0.01543,0.00782,0.01111,0.01028,0.01193,0.02283,0.01604,0.01316,0.02180,0.01625,0.01872,0.01213,0.01070,0.02406,0.01234,0.01440,0.00514,0.01604,0.01460,0.02098,0.01070,0.01728,0.01851,0.00864,0.01172,0.01892,0.01933,0.01419,0.01296,0.00967,0.01337,0.01337,0.01851,0.01131,0.01419,0.01090,0.02612,0.01275,0.01522,0.02365,0.02962,0.01789,0.01625,0.01234,0.01604,0.01687,0.02077,0.03949,0.00864,0.00596,0.00926,0.00596,0.01974,0.02489,0.03126,0.02036,0.02242,0.02468,0.00391,0.00000,0.00144
3,0.0,1915.0,0.01775,0.02245,0.01619,0.00992,0.01567,0.01358,0.00940,0.01723,0.02402,0.02245,0.02507,0.02924,0.02089,0.02141,0.01723,0.01932,0.02141,0.00679,0.02245,0.00522,0.01358,0.00418,0.01410,0.00574,0.01201,0.00992,0.00366,0.02402,0.02663,0.02872,0.00992,0.02350,0.00522,0.01619,0.00836,0.02037,0.01358,0.02089,0.00731,0.02141,0.00888,0.01567,0.01253,0.02298,0.01358,0.00992,0.00888,0.00783,0.00679,0.03133,0.04282,0.00627,0.00261,0.00261,0.00366,0.01410,0.01671,0.03760,0.01932,0.03029,0.03446,0.00261,0.00157,0.00000
4,0.0,22831.0,0.02816,0.01371,0.00767,0.03679,0.01380,0.00548,0.00473,0.02076,0.02716,0.00867,0.01310,0.02773,0.02803,0.00508,0.00920,0.02965,0.02878,0.00574,0.01572,0.01577,0.01007,0.00508,0.00604,0.00679,0.01205,0.03127,0.00775,0.00959,0.00797,0.02006,0.00359,0.00933,0.01191,0.01616,0.00788,0.02593,0.00854,0.01200,0.02098,0.02089,0.01367,0.01502,0.01809,0.02738,0.01796,0.01082,0.00705,0.01174,0.00858,0.03408,0.03964,0.00950,0.00429,0.00578,0.00604,0.01494,0.01734,0.04148,0.02483,0.03359,0.03679,0.00000,0.00044,0.00131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12057,1.0,1549.0,0.01679,0.03228,0.03486,0.00194,0.05939,0.04906,0.07037,0.00839,0.03551,0.02970,0.03034,0.01291,0.00839,0.01291,0.01097,0.00839,0.01743,0.03809,0.02841,0.00387,0.02582,0.02776,0.02518,0.00323,0.00452,0.00775,0.01679,0.01679,0.00323,0.01162,0.00968,0.00839,0.00129,0.00387,0.01420,0.01549,0.04067,0.03809,0.00710,0.00710,0.00775,0.03099,0.00323,0.01808,0.02195,0.00000,0.00000,0.00581,0.01485,0.01808,0.00129,0.00000,0.00129,0.01549,0.00000,0.00000,0.00000,0.00000,0.00323,0.01937,0.00581,0.00516,0.00000,0.02905
12058,0.0,8062.0,0.02270,0.01079,0.01191,0.01662,0.01240,0.00670,0.00521,0.01377,0.02493,0.01352,0.02282,0.02456,0.02394,0.01005,0.01228,0.01364,0.01563,0.01116,0.02543,0.00211,0.01488,0.01278,0.02406,0.00298,0.00955,0.01079,0.01302,0.02803,0.01129,0.02034,0.00546,0.01823,0.00136,0.01141,0.01067,0.02171

Por fim, agora vamos normalizar todos os atributos do conjunto de dados para o intervalo [0,1]. Aqui, faremos a normalização por re-escalonamento:

In [51]:
X = df.values

y = categories.values

normalizer = preprocessing.MinMaxScaler()

xscaled = normalizer.fit_transform(X)

Pronto! Agora o conjunto de dados está processado, em formato numérico, preparado para ser utilizado pelas técnicas de aprendizado de máquina.

In [52]:
xscaled

array([[0.00000000e+00, 2.91521417e-05, 1.27397366e-01, ...,
        1.29248198e-01, 1.95236236e-02, 0.00000000e+00],
       [0.00000000e+00, 1.38875530e-05, 2.09042594e-01, ...,
        1.39546859e-01, 2.65521281e-02, 0.00000000e+00],
       [0.00000000e+00, 1.13151328e-04, 1.52044982e-01, ...,
        2.01338826e-01, 0.00000000e+00, 1.34957826e-02],
       ...,
       [1.11111111e-01, 9.69491830e-05, 2.37772472e-01, ...,
        7.88877446e-01, 0.00000000e+00, 7.39456420e-01],
       [0.00000000e+00, 3.08857420e-03, 1.28321651e-01, ...,
        3.08959835e-02, 1.52284264e-02, 1.34957826e-02],
       [1.11111111e-01, 9.68026898e-05, 2.21905569e-01, ...,
        1.19464470e-01, 1.79617337e-02, 2.39550141e-01]])